In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

# 定义数据集
class TabularDataset(Dataset):
    def __init__(self, X, Y):
        self.x = X.values
        self.y = Y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])

# 定义模型
class TabularModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(14, 500)
        self.fc2 = nn.Linear(500, 100)
        self.fc3 = nn.Linear(100, 2)
        self.bn_in = nn.BatchNorm1d(14)
        self.bn1 = nn.BatchNorm1d(500)
        self.bn2 = nn.BatchNorm1d(100)

    def forward(self, x):
        x = self.bn_in(x)
        x = F.relu(self.fc1(x))
        x = self.bn1(x)

        x = F.relu(self.fc2(x))
        x = self.bn2(x)

        x = self.fc3(x)
        x = torch.sigmoid(x)

        return x

training_data = pd.read_csv('C:/Users/is_li/Desktop/paper/github/stock on MLOps/MLOps/tools/DVC/example/data/train.csv',
index_col= [0,1],header=[0,1])
valid_data = pd.read_csv('C:/Users/is_li/Desktop/paper/github/stock on MLOps/MLOps/tools/DVC/example/data/valid.csv',
index_col= [0,1],header=[0,1])
test_data = pd.read_csv('C:/Users/is_li/Desktop/paper/github/stock on MLOps/MLOps/tools/DVC/example/data/test.csv',
index_col= [0,1],header=[0,1])


c:\ProgramData\Anaconda3\envs\py380\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def getData(training_data,batch_size):
    '''
    获得数据集
    :param batch_size: 批大小
    :return: 获得相应的DataLoader
    '''

    # df = pd.read_csv("./data/adult.csv")
    df = training_data
    # 训练结果
    result_var = "label"
    # 分类型数据
    # cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex',
    #            'native-country']
    # 数值型数据
    cont_names = ['feature']

    # 将分类型数据转化为数字数据，并进行缺失数据填充
    def cat_names_change(df,cat_names):
        for col in df.columns:
            if col in cat_names:
                df[col].fillna("---")
                df[col] = LabelEncoder().fit_transform(df[col].astype(str))
            if col in cont_names:
                df[col] = df[col].fillna(0)
        return df

    # 分割训练数据和标签
    Y = df["label"]
    Y_label = LabelEncoder()
    Y = Y_label.fit_transform(Y)
    X = df.drop(columns=result_var)
    train_ds = TabularDataset(X, Y)
    # 使用DataLoader加载数据
    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    return train_dl

    print(Y)

def train(model, train_dl, DEVICE, epochs=20, learning_rate=0.01):
    '''
    训练

    :param model:
    :param train_dl:
    :param DEVICE:
    :param epochs:
    :param learning_rate:
    :return:
    '''

    model.train()
    writer = SummaryWriter(log_dir="runs/train_data")

    # 损失函数
    criterion = nn.CrossEntropyLoss()
    # 优化器
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # 记录损失
    losses = []
    for epoch in range(epochs):
        for i, (x, y) in enumerate(train_dl):
            x = x.float().to(DEVICE)
            y = y.long().to(DEVICE)
            # 清除梯度累积
            optimizer.zero_grad()
            # 前向计算
            outputs = model(x)
            # 计算损失
            loss = criterion(outputs, y)
            # 反向传播
            loss.backward()
            optimizer.step()

            losses.append(loss.cpu().data.item())

        writer.add_scalar("loss", loss.cpu().data.item(), global_step=epoch)
        test(model, train_dl, DEVICE, global_step=epoch)
        if (epoch + 1) % 10 == 0:
            print('Epoch : %d/%d,   Loss: %.4f' % (epoch + 1, epochs, np.mean(losses)))

    writer.close()

def test(model, train_dl, DEVICE, global_step=0):
    '''
    测试

    :param model:
    :param train_dl:
    :param DEVICE:
    :param global_step:
    :return:
    '''

    writer = SummaryWriter("runs/test_data")

    # 测试模型
    model.eval()
    correct = 0
    total = 0
    for i, (x, y) in enumerate(train_dl):
        x = x.float().to(DEVICE)
        y = y.long().to(DEVICE)
        outputs = model(x).cpu()
        pred = torch.argmax(outputs, dim=1)
        total += y.size(0)
        correct += (pred == y).sum()
    print('准确率: %.4f %%' % (100 * correct / total))
    writer.add_scalar("accurary", (100 * correct / total), global_step=global_step)
    writer.close()

if __name__ == "__main__":
    # 设置训练趟数
    epochs = 20
    learning_rate = 0.01

    # 准备设备
    DEVICE = torch.device("cpu")
    if torch.cuda.is_available():
        DEVICE = torch.device("cuda")
    print(DEVICE)
    # 获得数据集
    train_dl = getData(training_data,batch_size=64)
    # 实例化模型
    model = TabularModel().to(DEVICE)

    writer = SummaryWriter("runs/model")
    writer.add_graph(model, torch.randn(1, 14))
    writer.close()

    train(model, train_dl, DEVICE, epochs=epochs, learning_rate=learning_rate)


cuda
Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument weight in method wrapper__native_batch_norm)
Error occurs, No graph saved


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument weight in method wrapper__native_batch_norm)